- Given a lat lon
- Assume maximum temperature
- convert lat lon to correct projection
- join all times together
- from megacube extract that geospatial point
    - investgate interopolation vs. find nearest
- collapse time by mean
- return float

In [2]:
# GET /helloworld

print('hello world')


hello world


In [1]:
import datetime
import iris

iris.FUTURE.netcdf_promote = True

def load_data(pattern):
    # pattern is glob style
    cubes = iris.load('/opt/data/{}'.format(pattern))
    return iris.cube.CubeList(cubes).concatenate_cube()


ImportError: No module named 'iris'

In [7]:
import cartopy

def transform_coords(cube, lon, lat):
    expected_proj = cube.coords('projection_y_coordinate')[0].coord_system.as_cartopy_crs()
    given_proj = cartopy.crs.PlateCarree()
    return expected_proj.transform_point(lon, lat, given_proj)

ImportError: No module named 'cartopy'

In [8]:

def get_coords(cube, lat, lon):
    x, y = transform_coords(cube, lon, lat)
    first_year = next(cube.slices_over('time'))
    samples = [('projection_y_coordinate', y), ('projection_x_coordinate', x)]
    first_point = first_year.interpolate(samples, iris.analysis.Nearest())
    exact_x = first_point.coord('projection_x_coordinate')[0].points[0]
    exact_y = first_point.coord('projection_y_coordinate')[0].points[0]
    
    return(exact_x, exact_y)
    


NameError: name 'REQUEST' is not defined

In [10]:

def collapse_latlon(cube, x, y):
    x_const = iris.Constraint(projection_x_coordinate=x)
    y_const = iris.Constraint(projection_y_coordinate=y)
    return cube.extract(x_const).extract(y_const)



NameError: name 'REQUEST' is not defined

In [12]:

def extract_date(cube, year=2017, month=2, day=27, past_years=10):
    years = [year - i for i in range(past_years)]
    time_units = cube.coord('time').units
    vals = [time_units.date2num(datetime.datetime(y, month, day)) for y in years]
    time_constraint = iris.Constraint(time=vals)
    return cube.extract(time_constraint)



NameError: name 'REQUEST' is not defined

In [14]:

def mean(cube):
    return cube.collapsed('time', iris.analysis.MEAN)


NameError: name 'REQUEST' is not defined

In [2]:
patterns = {
    'temperature': 'maximum-temperature/*.nc',
    'rainfall': 'rainfall/*.nc'
}

In [4]:
# GET /:parameter/mean
import json

req = json.loads(REQUEST)

args = req['args']
lat = float(args['lat'][0])
lon = float(args['lon'][0])

path = req['path']
data_path = patterns[path['parameter']]

cube = load_data(data_path)
x, y = get_coords(cube, lat, lon)
time_cube = collapse_latlon(cube, x, y)
collapsed_time_cube = extract_date(time_cube)

val = float(mean(collapsed_time_cube).data)

response = {
    'value': val,
    'operation': 'mean',
    'start_date': '2007-02-27',
    'end_date': '2017-02-27'
}

print(response)

NameError: name 'REQUEST' is not defined

In [5]:
# ResponseInfo GET /:parameter/mean
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    }
}))

{"headers": {"Content-Type": "application/json"}}
